In [ ]:
import pandas as pd
import nltk
nltk.download('punkt_tab')

from data.game_data import GameData
from database.game_database import GameDatabase
from embedding.description_embedder import DescriptionEmbedder

API_KEY = "blah"
GAMES_FILE = r"C:\Users\kybur\OneDrive - Hochschule Luzern\Projects\DSPRO1\Data\clean_data.csv"

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kybur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Data

First we need to load the already cleaned and preprocessed data.

In [3]:
games = pd.read_csv(GAMES_FILE, delimiter=";")

They dates have to be formatted in ISO 8601 format.

In [ ]:
games["Release date"] = games["Release date"].apply(pd.to_datetime).dt.date
games["Release date"].apply(str)

C:\Users\kybur\AppData\Local\Temp\ipykernel_37116\2145479452.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  games["Release date"] = games["Release date"].apply(pd.to_datetime).dt.date


Any records that we don't want to store in the database should be filtered out now.

In [5]:
games

,Name,Release date,Estimated owners,About the game,Positive,Negative,Developers,Publishers,Categories,Genres,Tags,Popularity
0,Counter-Strike: Global Offensive,2012-08-21,100000000,Counter-Strike: Global Offensive (CS: GO) expa...,5764420,766677,"Valve,Hidden Path Entertainment",Valve,"Multi-player,Steam Achievements,Full controlle...","Action,Free to Play","FPS,Shooter,Multiplayer,Competitive,Action,Tea...",0.785569
1,Dota 2,2013-09-07,200000000,"The most-played game on Steam. Every day, mill...",1477153,300437,Valve,Valve,"Multi-player,Co-op,Steam Trading Cards,Steam W...","Action,Free to Play,Strategy","Free to Play,MOBA,Multiplayer,Strategy,e-sport...",0.566434
2,PUBG: BATTLEGROUNDS,2017-12-21,100000000,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",1154655,895978,"KRAFTON, Inc.","KRAFTON, Inc.","Multi-player,PvP,Online PvP,Stats,Remote Play ...","Action,Adventure,Free to Play,Massively Multip...","Survival,Shooter,Multiplayer,Battle Royale,FPS...",0.434255
3,Team Fortress 2,2007-10-10,100000000,The most fun you can have online - PC Gamer Is...,823693,56683,Valve,Valve,"Multi-player,Cross-Platform Multiplayer,Steam ...","Action,Free to Play","Free to Play,Hero Shooter,Multiplayer,FPS,Shoo...",0.275344
4,Black Myth: Wukong,2024-08-19,100000000,Black Myth: Wukong is an action RPG rooted in ...,663109,28700,Game Science,Game Science,"Single-player,Steam Achievements,Full controll...","Action,Adventure,RPG","Mythology,Action RPG,Action,RPG,Souls-like,Com...",0.257988
...,...,...,...,...,...,...,...,...,...,...,...,...
9574,Deserving Life,2017-10-13,100000,Deserving Life is a room-scale virtual reality...,96,20,"Lukas Paul, Gabriel Mittermair",Deserving Life,"Single-player,Steam Achievements",Free to Play,"Free to Play,Horror,VR",0.000211
9575,Timen runner,2017-05-26,100000,Hardcore pixel 2D platformer with steampunk el...,49,44,REX PEX GAMES,REX PEX GAMES,"Single-player,Steam Achievements,Steam Trading...","Casual,Indie","Indie,Casual,Platformer,2D,Pixel Graphics,Grea...",0.000211
9576,Mahjong Deluxe 2: Astral Planes,2016-07-20,100000,Mahjong Deluxe 2: Astral Planes is an out-of-t...,47,45,EnsenaSoft,EnsenaSoft,Unknown,Casual,"Casual,Puzzle",0.000211
9577,Pizza Frenzy Deluxe,2006-08-30,100000,Pizza Frenzy is a wacky action puzzler that pu...,125,5,"PopCap Games, Inc.","PopCap Games, Inc.",Single-player,Casual,Casual,0.000211


## Embedding
After preparing the data we can set up the transformer for the embedding. We are preparing several embedders that are domain specific to our data. For now we are using the `DescriptionEmbedder` which is a transformer that uses a `sentence-transformer` library to embed the description of our games.

In [6]:
embedder = DescriptionEmbedder()

Store the games and embedding in a data wrapper class for easier access later on. Doing so will use the embedder/transformer to create the embeddings from the data and determine the dimensionality of the embeddings.

In [7]:
game_data = GameData(games=games, embedder=embedder)

In [ ]:
embeddings = game_data.embeddings
dimension = game_data.embeddings[0].shape[0]
print(f"Embeddings dimension: {dimension}")

Embeddings shape: 768


## Database
Now, with the data prepare and the embeddings generated, we can store them in the database (currently using Pinecone).

In [17]:
db = GameDatabase(api_key=API_KEY, dimension=dimension)

In [ ]:
db.load_data(game_data.ids, game_data.metadata, game_data.embeddings)

Inserting records 1 to 100


PineconeException: UNKNOWN:Error received from peer  {grpc_message:"Error, message length too large: found 50299111 bytes, the limit is: 4194304 bytes", grpc_status:11, created_time:"2024-12-08T21:33:09.3577949+00:00"}

It takes some time until the data is created in the Pinecone service

In [33]:
db.describe_index()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

Once created, we can query the database to check if the data is stored correctly.

### Get By ID
A record can either be retrieved directly by ID...

In [21]:
game = db.get_by_id("Tomb Raider II")
print("Name:", game.metadata["Name"])

Name: Tomb Raider II


### Get By Embedding
...or we can get all the records that are similar to a given embedding.

In [21]:
matches = db.get_similar(game.values, k=10)
len(matches)

10

In [22]:
for match in matches:
    print("ID:", match.id, "- Name:", match.metadata["name"], ", Similarity:", match.score)

ID: Tomb Raider II - Name: Tomb Raider II , Similarity: 1.0000001
ID: Tomb Raider: Legend - Name: Tomb Raider: Legend , Similarity: 0.6332107
ID: Shadow of the Tomb Raider: Definitive Edition - Name: Shadow of the Tomb Raider: Definitive Edition , Similarity: 0.6131774
ID: Rise of the Tomb Raider - Name: Rise of the Tomb Raider™ , Similarity: 0.60820234
ID: Tomb Raider: Underworld - Name: Tomb Raider: Underworld , Similarity: 0.5933311
ID: Tomb Raider III - Name: Tomb Raider III , Similarity: 0.5847119
ID: Tomb Raider - Name: Tomb Raider , Similarity: 0.57854533
ID: Tomb Raider IV: The Last Revelation - Name: Tomb Raider IV: The Last Revelation , Similarity: 0.5702852
ID: Tomb Raider I-III Remastered Starring Lara Croft - Name: Tomb Raider I-III Remastered Starring Lara Croft , Similarity: 0.5581841
ID: Tomb Raider: Anniversary - Name: Tomb Raider: Anniversary , Similarity: 0.5469401


In [ ]:
ids = db.get_ids()
len(ids)